In [ ]:
import os

import torch
import torch.nn as nn
import torch.utils.data as tdata
import torchvision
from torch.nn import functional as F
import matplotlib.pyplot as plt

device = "mps"
torch.manual_seed(127)


# Data import, exploration and pre-processing

In [ ]:
class RoadsDataset(tdata.Dataset):
    root: str
    num_images: int
    images: list[torch.Tensor]
    gt_images: list[torch.Tensor]

    def __init__(self, root: str, num_images=20, transform=None, target_transform=None):
        self.root = root
        self.transform = transform
        self.target_transform = target_transform
        self.num_images = num_images
        assert 10 <= num_images <= 100
        self.images = []
        self.gt_images = []
        for i in range(num_images):
            image_path = os.path.join(self.root, "images/satImage_%.3d.png" % (i + 1))
            self.images.append(
                torchvision.io.read_image(image_path).type(torch.float32).to(device)
            )
            gt_image_path = os.path.join(
                self.root, "groundtruth/satImage_%.3d.png" % (i + 1)
            )
            self.gt_images.append(
                torch.movedim(
                    F.one_hot(
                        torch.div(
                            torch.squeeze(torchvision.io.read_image(gt_image_path)),
                            255,
                        ).type(torch.int64),
                        2,
                    ),
                    2,
                    0,
                ).to(dtype=torch.float32, device=device)
            )

        print("Loaded {} images from {}".format(num_images, root))

    def __len__(self):
        return self.num_images

    def __getitem__(self, item: int) -> tuple[torch.Tensor, torch.Tensor]:
        if self.transform:
            image = self.transform(self.images[item])
        else:
            image = self.images[item]

        if self.target_transform:
            gt_image = self.target_transform(self.gt_images[item])
        else:
            gt_image = self.gt_images[item]

        return image, gt_image

    def convert_gt_to_one_hot(self):
        pass

In [ ]:
training_data = RoadsDataset(root="data/training", num_images=30)
training_dataloader = tdata.DataLoader(training_data, batch_size=5, shuffle=True)

In [ ]:
X, _ = training_data[0]
X = X.to(device="cpu") / 255.0
# plt.subplot(121)
plt.imshow(torch.movedim(X, 0, 2))
# plt.subplot(122)
# plt.imshow(torch.movedim(Y,0,2), cmap="grayscale")
# plt.tight_layout()

# UNet model

In [ ]:
class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.double_conv(x)


class Down(nn.Module):
    """Downscaling with maxpool then double conv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2), DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Up(nn.Module):
    """Upscaling then double conv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.up = nn.ConvTranspose2d(
            in_channels, in_channels // 2, kernel_size=2, stride=2
        )
        self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        # input is CHW
        # diffY = x2.size()[2] - x1.size()[2]
        # diffX = x2.size()[3] - x1.size()[3]
        # x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2, diffY // 2, diffY - diffY // 2])
        # if you have padding issues, see
        # https://github.com/HaiyongJiang/U-Net-Pytorch-Unstructured-Buggy/commit/0e854509c2cea854e247a9c615f175f76fbb2e3a
        # https://github.com/xiaopeng-liao/Pytorch-UNet/commit/8ebac70e633bac59fc22bb5195e513d5832fb3bd
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)


class UNet(nn.Module):
    def __init__(self, n_channels, n_classes):
        super().__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes

        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        self.down4 = Down(512, 1024)
        self.up1 = Up(1024, 512)
        self.up2 = Up(512, 256)
        self.up3 = Up(256, 128)
        self.up4 = Up(128, 64)
        self.outc = OutConv(64, n_classes)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits


unet_model = UNet(n_channels=3, n_classes=2).to(device)

# Model training

In [ ]:
loss_fun = nn.BCELoss()
optimizer = torch.optim.Adam(unet_model.parameters(), lr=1e-3)


def train(dataloader: tdata.DataLoader, model: nn.Module, loss_fun, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch_num, (X_batch, Y_batch) in enumerate(dataloader):
        X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)
        # Compute prediction error
        pred = model(X_batch)
        loss = loss_fun(pred, Y_batch)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss, current = loss.item(), batch_num * len(X_batch)
        print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(training_dataloader, unet_model, loss_fun, optimizer)

print("Done training!")
torch.save(unet_model.state_dict(), "unet_model.pth")
print("Saved PyTorch Model State to unet_model.pth")
